In [1]:
!pip install langchain mistralai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.0/260.0 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.2/411.2 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: requests-toolbelt
    Found existing installation: requests-toolbelt 0.10.1
    Uninstalling requests-toolbelt-0.10.1:
      Successfully uninstalled requests-toolbelt-0.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following de

In [2]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.2 MB/s eta 0:00:00a 0:00:01


In [3]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 6.0 MB/s eta 0:00:0000:01


In [4]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of opentelemetry-sdk to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 15.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 54.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 74.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 94.9 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.1

In [5]:
# import os

# # Define the path for the vector_db directory
# vector_db_path = "/kaggle/working/vector_db"

# # Create vector_db if it does not exist
# if not os.path.exists(vector_db_path):
#     os.makedirs(vector_db_path)
#     print(f"Directory created: {vector_db_path}")
# else:
#     print(f"Directory already exists: {vector_db_path}")


In [6]:
import os
import json
from pathlib import Path
from dotenv import load_dotenv
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
from langchain.llms.base import LLM
from mistralai import Mistral, UserMessage
import argparse

# Initialize Mistral client with hardcoded API key (use Kaggle Secrets to store API keys securely)
api_key = "3gL9NenXUw9L12ZE4OzilERBWnURExey"  # Replace with your actual key or use Kaggle's secret management
model = "mistral-large-latest"
client = Mistral(api_key=api_key)

class MistralLLM(LLM):
    def _call(self, prompt: str, stop=None):
        messages = [UserMessage(content=prompt)]
        response = client.chat.complete(model=model, messages=messages)
        return response.choices[0].message.content.strip()

    @property
    def _identifying_params(self):
        return {"model": model}

    @property
    def _llm_type(self):
        return "mistral"
        
# Configuration for embeddings
stories_folder = Path("/kaggle/input/stories")  # Adjust path as needed
vector_db_path = "/kaggle/working/vector_db"
embedder = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


/tmp/ipykernel_23/3607933038.py:34: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedder = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [21]:
os.makedirs(vector_db_path, exist_ok=True)

# Function to compute embeddings
def compute_embeddings():
    if not stories_folder.exists():
        print(f"Directory not found: {stories_folder}")
        return
    
    documents = []
    for file_path in stories_folder.glob("*.txt"):
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
            documents.append(Document(page_content=content, metadata={"source": str(file_path)}))
    
    vector_store = Chroma.from_documents(documents, embedder, persist_directory=vector_db_path)
    vector_store.persist()
    print(f"Embeddings computed and saved to {vector_db_path}")

# Function to retrieve character information
def get_character_info(full_name: str):
    if not Path(vector_db_path).exists():
        print(f"Vector database not found: {vector_db_path}. Please run 'compute_embeddings' first.")
        return

    vector_store = Chroma(persist_directory=vector_db_path, embedding_function=embedder)
    retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})

    # Ensure that the query includes both first name and surname
    query = f"Provide structured details about the character with the full name '{full_name}' in JSON format. Include name, storyTitle, summary, relations, and characterType."

    related_docs = retriever.get_relevant_documents(query)
    context = "\n\n".join([doc.page_content for doc in related_docs if full_name.lower() in doc.page_content.lower()])

    if not context.strip():
        print(f"No matching content found for character '{full_name}'.")
        return

    prompt = f"Context:\n{context}\n\n{query}"
    response = mistral_llm(prompt)

    try:
        structured_response = json.loads(response)
        print(json.dumps(structured_response, indent=4))
    except json.JSONDecodeError:
        print(f"Could not parse response into JSON format. Response: {response}")


In [22]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Create text input for character name (initially hidden)
character_name_input = widgets.Text(
    value='',
    placeholder='Type the character name here',
    description='Character:',
    disabled=False
)
character_name_input.layout.display = 'none'  # Hide the input initially

# Button to trigger embeddings computation
compute_button = widgets.Button(description="Compute Embeddings")

# Button to trigger character information retrieval (initially hidden)
get_info_button = widgets.Button(description="Get Character Info")
get_info_button.layout.display = 'none'  # Hide the button initially

# Output area for displaying results
output_area = widgets.Output()

def on_compute_clicked(b):
    with output_area:
        clear_output()
        print("Computing embeddings...")
        compute_embeddings()
        print("Embeddings computation done.")
        # Show character input and get info button after embeddings are computed
        character_name_input.layout.display = None  # Unhide the input
        get_info_button.layout.display = None  # Unhide the button

def on_get_info_clicked(b):
    with output_area:
        clear_output()
        character_name = character_name_input.value
        if character_name:
            print(f"Retrieving info for {character_name}...")
            get_character_info(character_name)
            print("Character info retrieval done.")
        else:
            print("Please enter a character name.")

# Attach the click event handlers to the buttons
compute_button.on_click(on_compute_clicked)
get_info_button.on_click(on_get_info_clicked)

# Display the widgets
display(compute_button, character_name_input, get_info_button, output_area)


Button(description='Compute Embeddings', style=ButtonStyle())

Text(value='', description='Character:', layout=Layout(display='none'), placeholder='Type the character name h…

Button(description='Get Character Info', layout=Layout(display='none'), style=ButtonStyle())

Output()